<a href="https://colab.research.google.com/github/mostly-sunny/digital-health-hackathon/blob/main/train_deep_coxph_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CoxPH Model을 활용한 유전자 별 치료 효능률 계산 

## Requirements

### Main Library
- pycox
- pytorch
- numpy
- pandas
- torchtuples

### Input File
- **all-in-one-modified.csv** (from eliminate_invalid_data.ipynb)
- **virtual-data.csv** (from make_virtual_data.ipynb)

In [1]:
pip install pycox

     |████████████████████████████████| 73 kB 1.2 MB/s 
     |████████████████████████████████| 41 kB 682 kB/s 
     |████████████████████████████████| 65 kB 2.7 MB/s 
     |████████████████████████████████| 126 kB 7.8 MB/s 
     |████████████████████████████████| 1.9 MB 12.4 MB/s 
     |████████████████████████████████| 2.2 MB 28.2 MB/s 
     |████████████████████████████████| 357 kB 52.3 MB/s 


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn_pandas import DataFrameMapper
import pandas as pd

import torch
import torchvision
import torchtuples as tt

from pycox.models import CoxPH
from pycox.evaluation import EvalSurv

import random

- Cuda (GPU) 가 사용 가능한지 확인

In [3]:
torch.cuda.is_available()

True

# 재현성
- 코드의 재현성을 위한 랜덤 시드 고정
- 단, CPU에서 실행했을 때와 GPU에서 실행했을 때에는 결과가 달라지기 때문에, 동일하게 GPU에서만 학습을 진행

In [4]:
random.seed(123456)
np.random.seed(123456)
torch.manual_seed(123456)

# 파일 설명

### all-in-one-modified.csv (from eliminate_invalid_data.ipynb) 
- 유전자 변이 유무, 임상 변수, 생존 기간, 사망 여부, 치료 유무를 가지고 있는 환자 데이터
- G1 ~ G300, Var1 ~ Var10, Treatment, time, event
- 주어진 조건 (0 <= 임상변수 <= 9)에 맞지 않는 임상변수 값을 가진 환자는 제거
- 생존기간인 time이 음수인 값을 가진 환자 또한 제거

### virtual-data.csv (from make_virtual_data.ipynb) 
- 치료 효능률을 계산하기 위한 데이터
- (0번째 행) : 유전자 변이 모두 0, 임상 변수 모두 0, 치료 0
- (1번째 행) : 유전자 변이 모두 0, 임상 변수 모두 0, 치료 1
- (2~301번째 행) : 유전자 변이 n-1에만 1, 임상 변수 모두 0, 치료 0
- (302~601번재 행) : 유전자 변이 n-301에만 1, 임상 변수 모두 0, 치료 1

In [5]:
dataset = pd.read_csv('/content/all-in-one-modified.csv')
dataset_for_hr = pd.read_csv('/content/virtual-data.csv')

- all-in-one-modified.csv로부터 읽어들인 데이터 확인

In [6]:
dataset.head()

,Unnamed: 0,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20,G21,G22,G23,G24,G25,G26,G27,G28,G29,G30,G31,G32,G33,G34,G35,G36,G37,G38,G39,...,G274,G275,G276,G277,G278,G279,G280,G281,G282,G283,G284,G285,G286,G287,G288,G289,G290,G291,G292,G293,G294,G295,G296,G297,G298,G299,G300,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,Treatment,time,event
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,5,1,1,4,6,5,2,1,0,1,0,57.448331,1
1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,1,3,9,1,1,2,4,0,1,0,27.004439,1
2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,5,3,4,3,3,3,2,2,3,1,43.770511,1
3,3,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,2,7,2,3,5,0,1,4,5,3,1,32.281018,1
4,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,3,0,0,2,2,6,3,3,1,0,44.559284,0


# 데이터 비율 설정

### 테스트 데이터
- 전체 데이터(966개) 中 30% (290개)

### 훈련 데이터와 검증 데이터
- 전체 데이터(966개)에서 테스트 데이터(290개)를 뺀 (676개) 
- 그 中 훈련데이터 80% (541개) , 검증 데이터 20% (135개)

### 재현성
- 결과의 재현성을 위해 전체 데이터로부터 샘플링할 때 동일한 랜덤 시드를 사용하도록 함.

In [7]:
dataset_test = dataset.sample(frac=0.3, random_state = np.random.seed(123456))
dataset_train = dataset.drop(dataset_test.index)
dataset_val = dataset_train.sample(frac=0.2, random_state = np.random.seed(123456))
dataset_train = dataset_train.drop(dataset_val.index)

# 데이터 전처리

### columns_standardize
- 임상변수 : 0~9 사이의 값을 MinMaxScaler로 Scaling

### columns_leave
- 유전자 변이 유무 + 치료 유무 :0과 1로 표현돼 있기 때문에 그대로 사용

### DataFrameMapper
- pandas.DataFrame에서 원하는 열을 뽑아서 리스트로 만들어주는 Object
- 리스트로 만들때 Scaler 함수가 포함된 열은 Scaler 적용
- None이면 값 변환 없이 그대로 적용

In [8]:
columns_standardize = ['Var' + str(i) for i in range(1,11)]
columns_leave = ['G' + str(i) for i in range(1,301)]
columns_leave += ['Treatment']

standardize = [([col], MinMaxScaler()) for col in columns_standardize]

leave = [(col, None) for col in columns_leave]

x_mapper = DataFrameMapper(leave + standardize)

- 위에서 만든 DataFrameMapper로 DataFrame 중 입력 데이터를 모델이 학습할 수 있게 끔 리스트 형식으로 바꾸어 준다.

In [9]:
x_train = x_mapper.fit_transform(dataset_train).astype('float32')
x_val = x_mapper.transform(dataset_val).astype('float32')
x_test = x_mapper.transform(dataset_test).astype('float32')
x_for_hr = x_mapper.transform(dataset_for_hr).astype('float32')

- DataFrame에서 출력 데이터인 time(생존시간)과 event(사망여부)를 뽑아 출력 데이터를 추린다.
- 검증(Validation)을 위한 입력, 출력 세트 val을 만든다.

In [10]:
get_target = lambda df: (df['time'].values, df['event'].values)
y_train = get_target(dataset_train)
y_val = get_target(dataset_val)

durations_test, events_test = get_target(dataset_test)
val = x_val, y_val

# 딥러닝 네트워크 설계

### 함수 make_net
- pytorch network를 생성하여 반환하는 함수
- 매개변수 : input의 노드 수, output의 노드 수, 은닉층 수, 은닉층 노드 수, 은닉층 노드 감소 비율
- 활성화 함수로 ReLU 함수 사용
- 매 층마다 BatchNorm1d로 정규화
- 은닉층 수는 최소 1개부터 최대 4개까지 설정 가능.
- 은닉층 노드 감수 비율(rate)은 매 층마다 한 번씩 더 곱해짐.
- Dropout 비율은 0.2로 고정

예시) make_net(311, 1, 3, 1000, 0.5)
- 은닉층은 총 3층
- 첫번째 은닉층 노드 수 : 1000
- 두번째 은닉층 노드 수 : 500
- 세번째 은닉층 노드 수 : 250

In [11]:
def make_net(in_features, out_features, hidden, nodes, rate):
  if hidden == 1:
    network =  torch.nn.Sequential(
      torch.nn.Linear(in_features, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.2),

      torch.nn.Linear(nodes, out_features)
    )
  elif hidden == 2:
    network =  torch.nn.Sequential(
      torch.nn.Linear(in_features, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.2),

      torch.nn.Linear(nodes, int(nodes * rate)),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(int(nodes * rate)),
      torch.nn.Dropout(0.2),
            
      torch.nn.Linear(int(nodes * rate), out_features)
    )
  elif hidden == 3:
    network =  torch.nn.Sequential(
      torch.nn.Linear(in_features, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.2),

      torch.nn.Linear(nodes, int(nodes * rate)),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(int(nodes * rate)),
      torch.nn.Dropout(0.2),

      torch.nn.Linear(int(nodes * rate), int(nodes * (rate ** 2))),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(int(nodes * (rate ** 2))),
      torch.nn.Dropout(0.2),
            
      torch.nn.Linear(int(nodes * (rate ** 2)), out_features)
    )
  elif hidden == 4:
    network =  torch.nn.Sequential(
      torch.nn.Linear(in_features, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.2),

      torch.nn.Linear(nodes, int(nodes * rate)),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(int(nodes * rate)),
      torch.nn.Dropout(0.2),

      torch.nn.Linear(int(nodes * rate), int(nodes * (rate ** 2))),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(int(nodes * (rate ** 2))),
      torch.nn.Dropout(0.2),
      
      torch.nn.Linear(int(nodes * (rate ** 2)), int(nodes * (rate ** 3))),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(int(nodes * (rate ** 3))),
      torch.nn.Dropout(0.2),

      torch.nn.Linear(int(nodes * (rate ** 3)), out_features)
    )
  return network

# 결과 저장을 위한 DataFrame
- 공통적으로 Network의 설정과 평가지표인 C-index, Brier score가 함께 저장된다.


### result_treat_ratio
- 어떤 유전자 변이에 대한 치료효능률을 저장하는 DataFrame

In [12]:
columns_list = ['c index', 'brier', 'layers', 'nodes', 'rate', 'lr']
for i in range(1, 301):
  columns_list.append('G' + str(i))
result_treat_ratio = pd.DataFrame(columns = columns_list)

In [13]:
result_treat_ratio

,c index,brier,layers,nodes,rate,lr,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20,G21,G22,G23,G24,G25,G26,G27,G28,G29,G30,G31,G32,G33,G34,...,G261,G262,G263,G264,G265,G266,G267,G268,G269,G270,G271,G272,G273,G274,G275,G276,G277,G278,G279,G280,G281,G282,G283,G284,G285,G286,G287,G288,G289,G290,G291,G292,G293,G294,G295,G296,G297,G298,G299,G300


# 학습

### 변수 설명
- in_features : 입력데이터의 개수 (x_train.shape : 311 = 300(유전자) + 10(임상변수) + 1(치료유무))
- out_features : 출력노드의 개수
- layers_list : 은닉층 수를 가지고 있는 리스트
- nodes_list : 은닉층 노드 수를 가지고 있는 리스트
- rate_list : 은닉층 노드 감수 비율을 가지고 있는 리스트
- lr_list : 학습률을 가지고 있는 리스트
- total_num과 count : 학습 진행도를 확인하기 위한 변수

### 학습 모델
- layer, node, rate, learning rate에 따른 모델 생성
- layer: 1, 2, 3 (본 코드에서는 layer가 1개인 모델만 생성)
- node: 100부터 4000까지 100단위로 총 40개
- rate: 1, 1/2, 1/4
- learning rate(lr): 0.0001, 0.001, 0.01과 pycox에서 제공하는 최적 학습률 탐색 함수 사용 (총 4개)
- 총 1120개의 모델 생성(layer가 1일때는 rate가 고려되지 않음)

### 치료 효능률 계산 및 결과 저장
1. 훈련된 모델에 predict 함수를 통해 virtual-data.csv의 데이터를 입력
2. 각각의 가상 환자에 대한 위험비 도출
3. 300개 중 하나의 동일한 유전자 변이를 가졌으나 치료 유무만 다른 두 환자의 위험비의 비율 (치료효능률)을 2번으로부터 계산
4. 계산된 치료효능률을 result_treat_ratio의 저장
5. 저장 시 해당 모델의 네트워크 설정과 Brier Score와 C-index 함께 저장

In [14]:
in_features = x_train.shape[1]
out_features = 1

# repeat same process for layers_list = [1, 2, 3]
# this code only run for 1 hidden layer.
layers_list = [1]
nodes_list = list(range(100, 4001, 100))
rate_list = [1]
# rate_list = [1, (1/2), (1/4)] (when layer = 2, layer = 3)
lr_list = [0.0001,0.001,0.01, 0]

total_num = len(layers_list) * len(nodes_list) * len(rate_list) * len(lr_list)
count = 1


for rate in rate_list:  
  for nodes in nodes_list:
    for layers in layers_list:
      for lr in lr_list:
        print(count, '/', total_num)
        print('layers:', layers, ', nodes:', nodes, ',rate:', rate, ',lr:', lr)
        count += 1

        # manage reproducibility
        random.seed(123456)
        np.random.seed(123456)
        torch.manual_seed(123456)

        # make model
        net = make_net(in_features, out_features, layers, nodes, rate)
        model = CoxPH(net, tt.optim.Adam)
        batch_size = len(dataset_train)

        # set learning rate
        if lr == 0:
          lrfinder = model.lr_finder(x_train, y_train, batch_size, tolerance = 10)
          model.optimizer.set_lr(lrfinder.get_best_lr())
        else:
          model.optimizer.set_lr(lr)
        
        epochs = 512
        callbacks = [tt.callbacks.EarlyStopping()]
        verbose = True

        # train
        %%time
        model.fit(x_train, y_train, batch_size, epochs, callbacks, verbose, val_data=val, val_batch_size=batch_size)
        
        # compute after train
        _ = model.compute_baseline_hazards()
        surv = model.predict_surv_df(x_test)

        # evaluation (brier score, C-index)
        ev = EvalSurv(surv, durations_test, events_test, censor_surv='km')
        time_grid = np.linspace(durations_test.min(), durations_test.max(), 100)
        brier_score = ev.integrated_brier_score(time_grid)
        c_index = ev.concordance_td()

        # calculate hazard ratio
        log_partial_hazard = model.predict(x_for_hr)
        partial_hazard = [np.exp(lph) for lph in log_partial_hazard]

        # ratio with treated and untreated
        treat_ratio = []
        for gene in range(300):
          treat_ratio.append([partial_hazard[gene+302]/partial_hazard[gene+2],'G' + str(gene+1)])
        
        treat_ratio_info = [float(i[0]) for i in treat_ratio]
        
        # add result to Dataframe
        if lr == 0:
          info = [c_index, brier_score, layers, nodes, rate, lrfinder.get_best_lr()]
        else:
          info = [c_index, brier_score, layers, nodes, rate, lr]
        
        result_treat_ratio.loc[len(result_treat_ratio)] =  info + treat_ratio_info

1 / 160
layers: 1 , nodes: 100 ,rate: 1 ,lr: 0.0001
CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.2 µs
0:	[0s / 0s],		train_loss: 5.5934,	val_loss: 3.9221
1:	[0s / 0s],		train_loss: 5.5658,	val_loss: 3.9219
2:	[0s / 0s],		train_loss: 5.5285,	val_loss: 3.9218
3:	[0s / 0s],		train_loss: 5.5304,	val_loss: 3.9216
4:	[0s / 0s],		train_loss: 5.5337,	val_loss: 3.9214
5:	[0s / 0s],		train_loss: 5.4713,	val_loss: 3.9213
6:	[0s / 0s],		train_loss: 5.4877,	val_loss: 3.9211
7:	[0s / 0s],		train_loss: 5.4305,	val_loss: 3.9209
8:	[0s / 0s],		train_loss: 5.4599,	val_loss: 3.9207
9:	[0s / 0s],		train_loss: 5.4323,	val_loss: 3.9205
10:	[0s / 0s],		train_loss: 5.4187,	val_loss: 3.9203
11:	[0s / 0s],		train_loss: 5.3636,	val_loss: 3.9201
12:	[0s / 0s],		train_loss: 5.3659,	val_loss: 3.9198
13:	[0s / 0s],		train_loss: 5.3668,	val_loss: 3.9196
14:	[0s / 0s],		train_loss: 5.3876,	val_loss: 3.9194
15:	[0s / 0s],		train_loss: 5.3464,	val_loss: 3.9192
16:	[0s / 0s],		train_loss: 5.3677,	val_loss: 3

/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
3:	[0s / 0s],		train_loss: 5.0246,	val_loss: 3.9101
4:	[0s / 0s],		train_loss: 4.9513,	val_loss: 3.9062
5:	[0s / 0s],		train_loss: 4.8437,	val_loss: 3.9022
6:	[0s / 0s],		train_loss: 4.7740,	val_loss: 3.8977
7:	[0s / 0s],		train_loss: 4.6937,	val_loss: 3.8929
8:	[0s / 0s],		train_loss: 4.6493,	val_loss: 3.8880
9:	[0s / 0s],		train_loss: 4.5717,	val_loss: 3.8828
10:	[0s / 0s],		train_loss: 4.5104,	val_loss: 3.8773
11:	[0s / 0s],		train_loss: 4.4399,	val_loss: 3.8718
12:	[0s / 0s],		train_loss: 4.3812,	val_loss: 3.8663
13:	[0s / 0s],		train_loss: 4.3372,	val_loss: 3.8608
14:	[0s / 0s],		train_loss: 4.2795,	val_loss: 3.8553
15:	[0s / 0s],		train_loss: 4.2377,	val_loss: 3.8499
16:	[0s / 0s],		train_loss: 4.2175,	val_loss: 3.8444
17:	[0s / 0s],		train_loss: 4.1387,	val_loss: 3.8395
18:	[0s / 0s],		train_loss: 4.0858,	val_loss: 3.8349
19:	[0s / 0s],		train_loss: 4.0483,	val_loss: 3.8307
20:	[0s / 0s],		train_loss: 4.0059,	val_loss: 3.8276
21:	[0s / 0s],		t

# 결과 저장

### ratio_은닉층수_노드수_노드수.csv
- 계산한 치료효능률을 저장한 DataFrame을 csv파일로 변환

In [15]:
#name_ratio = '/content/' + 'ratio_' + str(layers_list[0]) + '_' + str(nodes_list[0]) + '_' + str(nodes_list[-1]) + '.csv'
name_ratio = 'c_index_ratio.csv'
result_treat_ratio.to_csv(name_ratio)

# 결과 확인
- Result DataFrame을 출력하여 결과 확인

In [16]:
result_treat_ratio

,c index,brier,layers,nodes,rate,lr,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20,G21,G22,G23,G24,G25,G26,G27,G28,G29,G30,G31,G32,G33,G34,...,G261,G262,G263,G264,G265,G266,G267,G268,G269,G270,G271,G272,G273,G274,G275,G276,G277,G278,G279,G280,G281,G282,G283,G284,G285,G286,G287,G288,G289,G290,G291,G292,G293,G294,G295,G296,G297,G298,G299,G300
0,0.516427,0.060031,1.0,100.0,1.0,0.000100,1.010559,0.967924,1.001722,0.984453,0.983554,0.954839,1.041509,0.972057,0.964119,1.014329,1.041670,0.948351,0.960239,0.950489,0.998895,0.964333,0.960420,0.975570,1.024754,0.989599,0.979770,0.985057,0.969861,0.988286,1.019862,0.989879,1.007333,0.974061,0.985415,0.993628,1.009887,0.957817,0.992215,0.957401,...,0.970707,0.997196,0.984385,0.961790,0.989746,0.965902,1.008646,0.998574,1.010530,0.983109,0.981918,0.979238,0.988246,1.000293,1.029220,0.974130,0.952003,0.949964,0.954556,0.991438,0.935317,1.019841,1.021423,1.018017,0.993340,0.953760,0.964421,0.999233,1.023145,0.989110,0.992190,0.953394,1.008802,0.976681,1.014205,0.947024,0.992719,0.972699,1.048026,0.960734
1,0.644281,0.057654,1.0,100.0,1.0,0.001000,0.765856,0.752127,0.801138,0.782794,0.783416,0.725605,0.819103,0.762286,0.771173,0.847613,0.791162,0.787264,0.730617,0.746629,0.779398,0.780964,0.765370,0.754102,0.832789,0.788928,0.764671,0.777253,0.757322,0.756116,0.751865,0.770858,0.769791,0.761699,0.802031,0.784130,0.795514,0.757639,0.760317,0.764172,...,0.766446,0.777434,0.742627,0.756668,0.785029,0.774640,0.787989,0.801836,0.799086,0.777049,0.761100,0.776043,0.737723,0.766887,0.766736,0.737776,0.752805,0.731697,0.754528,0.729778,0.723085,0.787591,0.778997,0.752925,0.743908,0.759124,0.774283,0.775248,0.797586,0.784878,0.771413,0.772372,0.812265,0.784505,0.803110,0.729637,0.777729,0.775353,0.819135,0.746716
2,0.662022,0.055987,1.0,100.0,1.0,0.010000,0.718393,0.754192,0.742440,0.736698,0.742548,0.740904,0.739995,0.755525,0.794280,0.753868,0.737246,0.756452,0.722898,0.726673,0.733112,0.745007,0.750340,0.757916,0.759137,0.757975,0.722553,0.773172,0.750799,0.723437,0.729775,0.763911,0.737649,0.760815,0.741199,0.753196,0.738466,0.730030,0.742737,0.754963,...,0.750273,0.734015,0.697338,0.735177,0.741727,0.720534,0.732952,0.758914,0.742966,0.727173,0.726949,0.761863,0.720880,0.729679,0.746231,0.743059,0.771978,0.732385,0.743550,0.694814,0.748670,0.715340,0.731808,0.710464,0.694503,0.741072,0.753655,0.734831,0.771514,0.743159,0.721331,0.753985,0.768126,0.739956,0.722317,0.713888,0.740590,0.727185,0.741267,0.734362
3,0.558972,0.059249,1.0,100.0,1.0,0.107227,0.874544,0.887460,0.914402,0.941099,0.903129,0.906635,0.935737,0.940186,0.929318,0.938768,0.904724,0.906621,0.899348,0.937582,0.897457,0.925446,0.860335,0.956939,0.867532,0.860995,0.888852,0.883971,0.914468,0.874452,0.986882,0.899678,0.904331,0.879029,0.840128,0.874968,0.930527,0.818148,0.957294,0.934662,...,0.933984,0.913093,0.881166,0.891561,0.867423,0.940801,0.942236,0.908667,0.917655,0.851931,0.962820,0.901337,0.892423,0.922524,0.933990,0.853891,0.971901,0.895537,0.804290,0.957049,0.903909,0.923071,0.890294,0.900147,0.889031,0.939768,0.932874,0.884760,0.968945,0.899722,0.897094,0.899203,0.887611,0.871017,0.900613,0.876597,0.902515,0.899791,0.924294,0.943178
4,0.572852,0.059133,1.0,200.0,1.0,0.000100,0.950062,0.896834,0.921132,0.911235,1.015061,0.867942,0.947888,0.920395,0.897357,0.960751,0.929760,0.953173,1.048333,0.922197,0.864756,0.931110,0.902810,0.922737,0.884101,0.985623,0.997316,0.993606,0.916786,1.024948,0.936992,0.957559,0.952428,0.962405,0.918045,0.907558,0.918250,0.999533,0.958741,0.863933,...,0.920208,0.867513,0.896328,0.965844,0.947764,0.986562,0.933570,0.896711,0.892678,0.928323,0.881854,0.961204,0.947028,0.949744,0.882682,0.941475,0.898506,0.917098,0.953521,0.889275,0.891454,0.927010,0.895053,0.913441,0.914745,0.976990,0.890262,0.970781,0.914297,1.009874,0.925594,0.921144,0.890475,0.904233,0.842527,0.898892,0.890058,0.878192,0.943655,0.940740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,